In [3]:
import assemblyai as aai
import os
from pydub import AudioSegment # Import pydub

# --- Configuration ---
# WARNING: Storing API keys directly in code is a security risk.
# It's highly recommended to use environment variables instead.
aai.settings.api_key = "503307250e13434b97068ef845a6a43a" 

# Set the path to your audio file.
FILE_PATH = '/Users/sujanh/Downloads/Gender and Politics Panel Discussion.mp3'

# --- Main Transcription and Export Logic ---
def transcribe_and_merge(file_path):
    """
    Transcribes an audio file, saves the full transcript, and merges all
    audio segments for each speaker into a single file per speaker.

    Args:
        file_path (str): The local path to the audio file.
    """
    if not os.path.exists(file_path):
        print(f"Error: File not found at '{file_path}'")
        return

    # Configure the transcription options.
    config = aai.TranscriptionConfig(speaker_labels=True)
    transcriber = aai.Transcriber()

    print(f"🎤 Starting transcription for '{file_path}'...")
    transcript = transcriber.transcribe(file_path, config)

    # --- Handle Transcription Errors ---
    if transcript.status == aai.TranscriptStatus.error:
        print(f"Transcription failed: {transcript.error}")
        return

    if not transcript.utterances:
        print("Diarization not available. The audio might be too short or have only one speaker.")
        if transcript.text:
            base_name, _ = os.path.splitext(os.path.basename(file_path))
            transcript_filename = f"{base_name}_transcript.txt"
            with open(transcript_filename, "w") as f:
                f.write(transcript.text)
            print(f"Full transcript saved to '{transcript_filename}'")
        return

    print("\n✅ Transcription Complete with Diarization.")
    
    # --- Save Full Transcript to a .txt file ---
    base_name, _ = os.path.splitext(os.path.basename(file_path))
    transcript_filename = f"{base_name}_transcript.txt"
    with open(transcript_filename, "w") as f:
        for utterance in transcript.utterances:
            f.write(f"Speaker {utterance.speaker}: {utterance.text}\n")
    print(f"📜 Full diarized transcript saved to '{transcript_filename}'")

    # --- NEW: Collect, Merge, and Export Audio Clips ---
    print("\n🔊 Collecting and merging audio clips for each speaker...")

    try:
        # Load the original audio file using pydub
        original_audio = AudioSegment.from_file(file_path)
    except Exception as e:
        print(f"Error loading audio file with pydub: {e}")
        print("Please ensure FFmpeg is installed and accessible in your system's PATH.")
        return

    # Create a directory to store the merged audio files
    output_dir = f"{base_name}_merged_speakers"
    os.makedirs(output_dir, exist_ok=True)
    
    # Use a dictionary to collect all audio segments for each speaker
    speaker_audio_segments = {}

    for utterance in transcript.utterances:
        speaker = utterance.speaker
        start_ms = utterance.start
        end_ms = utterance.end

        # Slice the audio segment for the current utterance
        clip = original_audio[start_ms:end_ms]

        # If the speaker is already in our dictionary, append the new clip
        if speaker in speaker_audio_segments:
            speaker_audio_segments[speaker] += clip
        # Otherwise, create a new entry for this speaker
        else:
            speaker_audio_segments[speaker] = clip
    
    # Now, export the merged audio for each speaker
    for speaker, merged_audio in speaker_audio_segments.items():
        # Define the output filename, e.g., "SPEAKER_A_merged.mp3"
        merged_filename = os.path.join(output_dir, f"SPEAKER_{speaker}_merged.mp3")
        
        print(f"  -> Exporting merged audio for Speaker {speaker} to '{merged_filename}'")
        merged_audio.export(merged_filename, format="mp3")
    
    print(f"\n✅ All merged audio files saved in the '{output_dir}' directory.")


# --- Run the script ---
if __name__ == "__main__":
    transcribe_and_merge(FILE_PATH)

🎤 Starting transcription for '/Users/sujanh/Downloads/Gender and Politics Panel Discussion.mp3'...

✅ Transcription Complete with Diarization.
📜 Full diarized transcript saved to 'Gender and Politics Panel Discussion_transcript.txt'

🔊 Collecting and merging audio clips for each speaker...
  -> Exporting merged audio for Speaker A to 'Gender and Politics Panel Discussion_merged_speakers/SPEAKER_A_merged.mp3'
  -> Exporting merged audio for Speaker B to 'Gender and Politics Panel Discussion_merged_speakers/SPEAKER_B_merged.mp3'
  -> Exporting merged audio for Speaker C to 'Gender and Politics Panel Discussion_merged_speakers/SPEAKER_C_merged.mp3'
  -> Exporting merged audio for Speaker D to 'Gender and Politics Panel Discussion_merged_speakers/SPEAKER_D_merged.mp3'
  -> Exporting merged audio for Speaker E to 'Gender and Politics Panel Discussion_merged_speakers/SPEAKER_E_merged.mp3'

✅ All merged audio files saved in the 'Gender and Politics Panel Discussion_merged_speakers' directory.